# Reading, inspecting, cleaning up, saving tabular data

I want to create ensembles of decision trees. It's of two key techniques in machine learning (the other being neural networks with stochastic gradient descent).

In [14]:
# First, let's find a dataset of tabular data.
# I'll choose the titanic dataset from kaggle.
import pathlib
import fastbook
import kaggle
import shutil

def get_kaggle_dataset(comp):
    kaggle_api_credentials = pathlib.Path('~/.kaggle/kaggle.json').expanduser().read_text()
    path = fastbook.URLs.path(comp)
    if path.exists():
        print(path, "already exists.")
        return path
    path.mkdir(parents=True)
    kaggle.api.competition_download_cli(comp, path=path)
    shutil.unpack_archive(str(path/f'{comp}.zip'), str(path))
    return path
    
print(get_kaggle_dataset("titanic"))


100%|██████████| 34.1k/34.1k [00:00<00:00, 68.1MB/s]


/home/john/.fastai/archive/titanic


In [16]:
path = get_kaggle_dataset("titanic")
print(path.ls())

/home/john/.fastai/archive/titanic already exists.
[Path('/home/john/.fastai/archive/titanic/gender_submission.csv'), Path('/home/john/.fastai/archive/titanic/titanic.zip'), Path('/home/john/.fastai/archive/titanic/train.csv'), Path('/home/john/.fastai/archive/titanic/test.csv')]


In [18]:
# Load the training set with pandas and inspect it.
import pandas as pd

df = pd.read_csv(path/'train.csv', low_memory=False)

In [19]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [23]:
[ (col, len(df[col].unique())) for col in df.columns ]


[('PassengerId', 891),
 ('Survived', 2),
 ('Pclass', 3),
 ('Name', 891),
 ('Sex', 2),
 ('Age', 89),
 ('SibSp', 7),
 ('Parch', 7),
 ('Ticket', 681),
 ('Fare', 248),
 ('Cabin', 148),
 ('Embarked', 4)]

In [24]:
[ (col, df[col].unique()) for col in df.columns if len(df[col].unique()) < 10 ]

[('Survived', array([0, 1])),
 ('Pclass', array([3, 1, 2])),
 ('Sex', array(['male', 'female'], dtype=object)),
 ('SibSp', array([1, 0, 3, 4, 2, 5, 8])),
 ('Parch', array([0, 1, 2, 5, 3, 4, 6])),
 ('Embarked', array(['S', 'C', 'Q', nan], dtype=object))]

In [62]:
# Q: How did sex and class affect rate of survival?

for sex in df["Sex"].unique():
    for c in df["Pclass"].unique():
        rows = df.loc[(df["Sex"] == sex) & (df["Pclass"] == c)]["Survived"]
        print(f"Class {c}", sex, f"{(sum(rows)/len(rows)):.2f}", f"({sum(rows)} / {len(rows)})")


Class 3 male 0.14 (47 / 347)
Class 1 male 0.37 (45 / 122)
Class 2 male 0.16 (17 / 108)
Class 3 female 0.50 (72 / 144)
Class 1 female 0.97 (91 / 94)
Class 2 female 0.92 (70 / 76)


In [81]:
# Over 90% of class 1 & class 2 females survived. Which didn't?
rows = df.loc[ (df["Sex"] == "female") & (df["Pclass"] < 3)]
rows.loc[ rows["Survived"] == 0 ]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
41,42,0,2,"Turpin, Mrs. William John Robert (Dorothy Ann Wonnacott)",female,27.0,1,0,11668,21.0000,NaN,S
177,178,0,1,"Isham, Miss. Ann Elizabeth",female,50.0,0,0,PC 17595,28.7125,C49,C
199,200,0,2,"Yrois, Miss. Henriette (""Mrs Harbeck"")",female,24.0,0,0,248747,13.0000,NaN,S
297,298,0,1,"Allison, Miss. Helen Loraine",female,2.0,1,2,113781,151.5500,C22 C26,S
312,313,0,2,"Lahtinen, Mrs. William (Anna Sylfven)",female,26.0,1,1,250651,26.0000,NaN,S
357,358,0,2,"Funk, Miss. Annie Clemmer",female,38.0,0,0,237671,13.0000,NaN,S
498,499,0,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0,1,2,113781,151.5500,C22 C26,S
772,773,0,2,"Mack, Mrs. (Mary)",female,57.0,0,0,S.O./P.P. 3,10.5000,E77,S
854,855,0,2,"Carter, Mrs. Ernest Courtenay (Lilian Hughes)",female,44.0,1,0,244252,26.0000,NaN,S


In [101]:
# Observation: 5/9 had NaN cabin. Almost all Embarked from S. Are these meaningful?
survived = len(df.loc[df["Survived"] == 1])
rate_all = survived / len(df)

na = df.loc[df['Cabin'].isna()]
na_survived = len(na.loc[na["Survived"] == 1]) 
rate_na = na_survived / len(na)

s = df.loc[df['Embarked'] == "S"]
s_survived = len(s.loc[s["Survived"] == 1])
rate_s = s_survived / len(s)

print(f"Survived all: {rate_all:.2f} ({survived}/{len(df)})")
print(f"Survived  na: {rate_na:.2f} ({na_survived}/{len(na)})")
print(f"Survived   s: {rate_s:.2f} ({s_survived}/{len(s)})")

print()
print(f"na: {rate_na / rate_all : .2f}")
print(f"s: {rate_s / rate_all: .2f} ")

# Lower rate of survival for "NA" cabins and for "S" embarkations than for overall population

Survived all: 0.38 (342/891)
Survived  na: 0.30 (206/687)
Survived   s: 0.34 (217/644)

na:  0.78
s:  0.88 
